## breast cancer prediction



In [0]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from tqdm import tqdm_notebook 
import seaborn as sns
import time
import torch

In [5]:
torch.manual_seed(0)

## preprocessing dataset

In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

dataset=pd.read_csv('data.csv')
print(dataset.shape)
data=dataset.iloc[:,2:-1].values
label=dataset.iloc[:,1]
print(label.shape)
print(data.shape)
label=le.fit_transform(label)


print(label.shape)

(569, 33)
(569,)
(569, 30)
(569,)


In [7]:
X_train, X_val, Y_train, Y_val = train_test_split(data, label, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, label.shape)

(455, 30) (114, 30) (569,)


## Using torch tensors and autograd

In [0]:
X_train, Y_train, X_val, Y_val = map(torch.tensor, (X_train, Y_train, X_val, Y_val))
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
epochs=1000

In [0]:
class Ffn(nn.Module):
  
  def __init__(self):
    super().__init__()
    torch.manual_seed(0)
    self.net = nn.Sequential(
        nn.Linear(30, 30), 
        nn.Sigmoid(), 
        nn.Linear(30,60), 
        nn.Softmax()
    )

  def forward(self, X):
    return self.net(X)
    

In [0]:
def accuracy(y_hat,Y_train):
  pred = torch.argmax(y_hat, dim=1)
  return (pred == Y_train).float().mean()

In [0]:
def loss(y_hat,Y):
  loss = F.cross_entropy(y_hat, Y)
  return loss

In [0]:
def fit(x,y,fn,loss,opt):
  for epoch in range(epochs):
    loss1 = loss(fn(x.float()), y)

    loss1.backward()
    opt.step()
    opt.zero_grad()
    
  return loss1.item()

In [21]:
fn = Ffn()
opt = optim.SGD(fn.parameters(), lr=1)
fit(X_train, Y_train, fn, loss,opt)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


3.485275983810425

In [22]:
device = torch.device("cuda")

X_train=X_train.to(device)
Y_train=Y_train.to(device)
fn = Ffn()
fn.to(device)
tic = time.time()
print('Final loss', fit(X_train, Y_train, fn, loss,opt))
toc = time.time()
print('Time taken', toc - tic)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Final loss 4.098293304443359
Time taken 0.962367057800293
